# Step 1 - Scraping

Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.

* Create a Jupyter Notebook file called `mission_to_mars.ipynb` and use this to complete all of your scraping and analysis tasks. The following outlines what you need to scrape.

### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

```python
# Example:
news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."
```

### JPL Mars Space Images - Featured Image

* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

```python
# Example:
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'
```

### Mars Weather

* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

```python
# Example:
mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'
```

### Mars Facts

* Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

### Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

```python
# Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]
```


In [43]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import os
import urllib.request

In [44]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser('chrome', **executable_path, headless=False)

In [45]:
nasa_url = "https://mars.nasa.gov/news"

In [46]:
browser.visit(nasa_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [47]:
# Print news title
news_title = soup.find("div", class_="content_title").text
print(news_title)

NASA's Treasure Map for Water Ice on Mars


In [48]:
# Print news paragraph
news_p = soup.find('div', class_="article_teaser_body").text
print(news_p)

A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.


In [49]:
#JPL Mars Space Images - Featured Image
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

In [50]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

image_name= soup.find('article', class_='carousel_item')['alt'] 
print(image_name)

Charon in Enhanced Color


In [51]:
first_url = 'https://www.jpl.nasa.gov'
img_url = soup.find(attrs={'data-title':image_name})["data-fancybox-href"] 
featured_image_url = first_url + img_url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19968_ip.jpg


In [52]:
# Mars Weather
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [53]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_weather= soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
print(mars_weather)

InSight sol 372 (2019-12-13) low -97.2ºC (-143.0ºF) high -21.2ºC (-6.2ºF)
winds from the SSE at 5.9 m/s (13.3 mph) gusting to 20.2 m/s (45.2 mph)
pressure at 6.60 hPapic.twitter.com/SXXKNyUaJu


In [54]:
# Mars Facts
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

# Use Pandas to convert the data to a HTML table string.
tables = pd.read_html(facts_url)

# create a dataframe to show the table
facts_df = tables[1]
facts_df

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-153 to 20 °C,-88 to 58°C


In [55]:
facts_df.to_html('facts_table.html')
!open facts_table.html

In [61]:
# Mars Hemispheres
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [62]:
product = soup.find('div', class_="collapsible results")
hemisphere_items = product.find_all('div', class_="item")
hemisphere_img_urls = []

for item in hemisphere_items:
    try:
        hemisphere_title  = item.find('h3').text
        hemisphere_href = item.find('a')['href']
        hemisphere_url = "https://astrogeology.usgs.gov"+hemisphere_href
        hemisphere_page = requests.get(hemisphere_url).text
        soup = BeautifulSoup(hemisphere_page, 'html.parser')

        hemisphere_page_img = soup.select('#wide-image > div > ul > li:nth-child(1) > a')
        hemisphere_img_url =  hemisphere_page_img[0]['href']
        hemisphere_img_dict = { "image title": hemisphere_title, "image url": hemisphere_img_url }
        hemisphere_img_urls.append(hemisphere_img_dict)

    except Exception as e:
        print(e)

hemisphere_img_urls       

[{'image title': 'Cerberus Hemisphere Enhanced',
  'image url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'image title': 'Schiaparelli Hemisphere Enhanced',
  'image url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'image title': 'Syrtis Major Hemisphere Enhanced',
  'image url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'image title': 'Valles Marineris Hemisphere Enhanced',
  'image url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]